In [65]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.mol_opt import MolOpt
from mol_opt.decoder_mol_opt import MolOptDecoder
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time
import torch.nn.functional as F

sys.argv = ["", "-cuda"]
args = get_args()
args.output_dir = "mol_opt/output/"
args.model_type = "molemb"




/home/octav/gitrepos/tum-thesis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
molopt = MolOpt(args).to(device = args.device)
molopt

moloptdecoder = MolOptDecoder(args).to(device = args.device)
moloptdecoder

MolOptDecoder(
  (fc1_SYMBOLS): Linear(in_features=150, out_features=150, bias=True)
  (fc2_SYMBOLS): Linear(in_features=150, out_features=64, bias=True)
  (fc1_CHARGES): Linear(in_features=150, out_features=150, bias=True)
  (fc2_CHARGES): Linear(in_features=150, out_features=5, bias=True)
  (fc1_BONDS): Linear(in_features=300, out_features=300, bias=True)
  (fc2_BONDS): Linear(in_features=300, out_features=5, bias=True)
)

In [67]:
molopt

MolOpt(
  (GCN): GCN(
    (W_message_i): Linear(in_features=100, out_features=200, bias=False)
    (W_message_h): Linear(in_features=200, out_features=200, bias=False)
    (W_message_o): Linear(in_features=293, out_features=150, bias=True)
    (W_mol_h): Linear(in_features=150, out_features=100, bias=True)
    (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
    (dropout_gcn): Dropout(p=0.0, inplace=False)
    (dropout_ffn): Dropout(p=0.0, inplace=False)
  )
  (opt0): Linear(in_features=150, out_features=14000, bias=True)
  (opt1): Linear(in_features=14000, out_features=10500, bias=True)
)

In [68]:
data_loader = get_loader("molgen/data/chembl50", "train", args.batch_size)
for i in data_loader:
    X = (MolGraph(i))
    Y = X
    break


In [69]:
x_embedding = molopt.encode(X)
x_opt = molopt.optimize(x_embedding, X)
moloptdecoder(x_opt, X, Y)

torch.Size([1, 25, 150])
torch.Size([1, 16, 150])
torch.Size([1, 29, 150])
torch.Size([1, 35, 150])
torch.Size([1, 21, 150])
torch.Size([1, 47, 150])
torch.Size([1, 18, 150])
torch.Size([1, 23, 150])
torch.Size([1, 23, 150])
torch.Size([1, 18, 150])
torch.Size([1, 16, 150])
torch.Size([1, 23, 150])
torch.Size([1, 18, 150])
torch.Size([1, 21, 150])
torch.Size([1, 7, 150])
torch.Size([1, 12, 150])
torch.Size([1, 12, 150])
torch.Size([1, 21, 150])
torch.Size([1, 17, 150])
torch.Size([1, 23, 150])
torch.Size([1, 25, 150])
torch.Size([1, 27, 150])
torch.Size([1, 30, 150])
torch.Size([1, 25, 150])
torch.Size([1, 23, 150])
torch.Size([1, 21, 150])
torch.Size([1, 14, 150])
torch.Size([1, 19, 150])
torch.Size([1, 32, 150])
torch.Size([1, 28, 150])
torch.Size([1, 24, 150])
torch.Size([1, 26, 150])
torch.Size([1, 17, 150])
torch.Size([1, 26, 150])
torch.Size([1, 20, 150])
torch.Size([1, 22, 150])
torch.Size([1, 43, 150])
torch.Size([1, 28, 150])
torch.Size([1, 13, 150])
torch.Size([1, 24, 150])
t

(tensor([[-0.1379,  0.0880, -0.0961,  ..., -0.0051, -0.1702, -0.1379],
         [-0.1612, -0.1327, -0.1081,  ..., -0.2362, -0.0267, -0.0861],
         [-0.1026, -0.0022, -0.1813,  ...,  0.0447,  0.0229, -0.2095],
         ...,
         [-0.1233, -0.0891, -0.2539,  ...,  0.1859, -0.1849, -0.1634],
         [-0.0999, -0.1696,  0.1227,  ..., -0.1062, -0.2433, -0.1976],
         [-0.1201, -0.0350, -0.1256,  ..., -0.1344, -0.1834, -0.2237]],
        device='cuda:0', grad_fn=<CatBackward>),
 tensor([[ 0.0747,  0.1814,  0.0343, -0.2024,  0.1619],
         [ 0.2482,  0.3247,  0.3161, -0.1004,  0.2377],
         [ 0.2086,  0.1960,  0.0558, -0.0123,  0.1792],
         ...,
         [ 0.2056,  0.0856,  0.0148, -0.0530,  0.0119],
         [ 0.3186,  0.1433, -0.0803,  0.0985,  0.2974],
         [ 0.3686,  0.3580, -0.0541, -0.1275, -0.0723]], device='cuda:0',
        grad_fn=<CatBackward>),
 tensor([[-1.0000e+03, -1.0000e+03, -1.0000e+03, -1.0000e+03,  1.0000e+03],
         [-1.0069e-01,  3.1747e-01

In [20]:
molopt.opt0(x_embedding)

tensor([[-1.8411,  0.3446, -0.6352,  ..., -1.2947,  0.7855,  4.2920],
        [-0.4773, -0.0084, -0.0781,  ..., -0.3271,  0.5665,  2.3345],
        [-1.8247, -0.1516, -0.2823,  ..., -0.3272,  0.8346,  6.0111],
        ...,
        [-1.4480,  0.3044, -0.2787,  ..., -0.1164,  1.0019,  2.6259],
        [-0.9277,  0.8908,  0.2876,  ..., -0.1627,  0.5290,  1.6644],
        [-1.1258,  0.2139,  0.2089,  ...,  0.1779,  0.6391,  2.7473]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [21]:
molopt.opt0(x_embedding)

tensor([[-1.8411,  0.3446, -0.6352,  ..., -1.2947,  0.7855,  4.2920],
        [-0.4773, -0.0084, -0.0781,  ..., -0.3271,  0.5665,  2.3345],
        [-1.8247, -0.1516, -0.2823,  ..., -0.3272,  0.8346,  6.0111],
        ...,
        [-1.4480,  0.3044, -0.2787,  ..., -0.1164,  1.0019,  2.6259],
        [-0.9277,  0.8908,  0.2876,  ..., -0.1627,  0.5290,  1.6644],
        [-1.1258,  0.2139,  0.2089,  ...,  0.1779,  0.6391,  2.7473]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [22]:
molopt.opt0

Linear(in_features=150, out_features=14000, bias=True)

In [23]:
layer = torch.nn.Linear(150, 150).to(device = args.device)

In [24]:
layer(x_embedding)

tensor([[ 2.5491, -3.8243, -1.9778,  ..., -3.0443,  0.4862,  1.3156],
        [ 0.3130, -2.2259, -0.7063,  ..., -1.8093,  0.6623,  0.5332],
        [ 1.2279, -4.6440, -2.0396,  ..., -3.8873,  1.6024,  0.6037],
        ...,
        [ 1.0914, -3.0059, -1.1852,  ..., -1.9341,  0.6392,  0.6749],
        [ 0.1556, -2.2109, -0.3331,  ..., -1.1977,  0.2608,  0.5255],
        [ 0.1799, -2.4665, -0.7200,  ..., -1.3656,  0.9785,  0.2966]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [25]:
x_embedding.transpose(0,1)

tensor([[ 2.4082,  2.1650,  3.2520,  ...,  2.4842,  2.0350,  2.0028],
        [-5.0467, -4.8617, -7.2811,  ..., -4.0614, -3.4622, -3.7260],
        [ 0.6564,  0.5581,  1.4251,  ...,  0.8704, -0.4082,  0.5978],
        ...,
        [-1.0022, -1.4396, -3.6880,  ..., -0.7175, -1.7962, -1.9513],
        [ 3.8293,  2.4059,  5.6922,  ...,  3.0192,  3.6203,  3.4795],
        [ 2.0169,  1.0477,  2.5341,  ...,  1.0565,  0.0688,  0.7988]],
       device='cuda:0', grad_fn=<TransposeBackward0>)

In [26]:
x_embedding[1,:17]

tensor([ 2.1650, -4.8617,  0.5581,  1.1072, -5.4497,  2.2931, -1.5579,  3.2176,
         1.7520, -4.2252, -2.9697, -1.3259, -3.2492, -3.0051, -0.2153, -2.6787,
        -7.8198], device='cuda:0', grad_fn=<SliceBackward>)

In [27]:
X.scope

[(0, 25),
 (25, 16),
 (41, 29),
 (70, 35),
 (105, 21),
 (126, 47),
 (173, 18),
 (191, 23),
 (214, 23),
 (237, 18),
 (255, 16),
 (271, 23),
 (294, 18),
 (312, 21),
 (333, 7),
 (340, 12),
 (352, 12),
 (364, 21),
 (385, 17),
 (402, 23),
 (425, 25),
 (450, 27),
 (477, 30),
 (507, 25),
 (532, 23),
 (555, 21),
 (576, 14),
 (590, 19),
 (609, 32),
 (641, 28),
 (669, 24),
 (693, 26),
 (719, 17),
 (736, 26),
 (762, 20),
 (782, 22),
 (804, 43),
 (847, 28),
 (875, 13),
 (888, 24),
 (912, 25),
 (937, 18),
 (955, 34),
 (989, 25),
 (1014, 29),
 (1043, 13),
 (1056, 41),
 (1097, 20),
 (1117, 16),
 (1133, 17)]

In [28]:
x_opt.narrow(0,2,1).narrow(1,0,17).shape

torch.Size([1, 17])

In [29]:
x_opt.shape

torch.Size([50, 10500])

In [30]:
x_opt.shape

torch.Size([50, 10500])

In [31]:
x_opt.narrow(1, 20, 20).shape

torch.Size([50, 20])

In [32]:
x_opt[0,:17].unsqueeze(0).shape

torch.Size([1, 17])

In [33]:
x_opt.shape

torch.Size([50, 10500])

In [34]:
molopt.opt0(molopt.mol_embs).shape

ModuleAttributeError: 'MolOpt' object has no attribute 'mol_embs'

In [35]:
molopt.mol_embs.shape

ModuleAttributeError: 'MolOpt' object has no attribute 'mol_embs'

In [36]:
molopt.opt0

Linear(in_features=150, out_features=14000, bias=True)

In [14]:
x_embedding.sum(axis = 0).shape

torch.Size([150])